In [1]:
from dwave.system.samplers import DWaveSampler
from minorminer import find_embedding
from dwave.system.composites import FixedEmbeddingComposite
import numpy as np
from scipy.sparse import dok_matrix

In [10]:
solver = "DW_2000Q_6"
sampler = DWaveSampler(solver=solver)

qubo_dict = {(0, 0): -0.25, (0, 1): 1, (1, 1): -0.25, (2, 2): -0.25}

offset0 = -0.1
offset1 = 0
tries = 100

In [11]:
qpu_graph = sampler.edgelist
anneal_offset = np.zeros(2048)
for _ in range(tries):
    embedding = find_embedding(qubo_dict, qpu_graph)
    embedding_idx = [idx for embed_list in embedding.values() for idx in embed_list]
    embed = FixedEmbeddingComposite(sampler, embedding)
    anneal_offset_ranges = np.array(
        embed.properties["child_properties"]["anneal_offset_ranges"]
    )
    offset_range = anneal_offset_ranges[embedding_idx]
    range0 = offset_range[0][0] <= offset0 <= offset_range[0][1]
    range1 = offset_range[1][0] <= offset1 <= offset_range[1][1]
    if range0 and range1:
        anneal_offset[embedding_idx[0]] = offset0
        anneal_offset[embedding_idx[1]] = offset1
        break
print(anneal_offset_ranges[embedding_idx])
print(embedding)
print(qubo_dict)

[[-0.13966355  0.11587289]
 [-0.13347227  0.13965449]
 [-0.14010183  0.04596959]]
{0: [1523], 1: [1527], 2: [1139]}
{(0, 0): -0.25, (0, 1): 1, (1, 1): -0.25, (2, 2): -0.25}


In [12]:
dwave_config = {
    "annealing_time": 10,  # integer microseconds [1, 2000]
    "answer_mode": "histogram",  # histogram or raw
    "auto_scale": False,
    "num_reads": 1000,  # raw will dump out all results
    "num_spin_reversal_transforms": 0,
    "readout_thermalization": 0,  # default 0
    "programming_thermalization": 1000,  # default 1000
    "chain_strength": 1,
    "anneal_offsets": list(anneal_offset),
}

In [13]:
result = embed.sample_qubo(qubo_dict, **dwave_config)
raw = result.to_pandas_dataframe()

In [14]:
raw

,0,1,2,chain_break_fraction,energy,num_occurrences
0,1,0,1,0.0,-0.50,613
1,0,1,1,0.0,-0.50,301
2,0,1,0,0.0,-0.25,18
3,1,0,0,0.0,-0.25,30
4,0,0,1,0.0,-0.25,33
5,0,0,0,0.0,0.00,4
6,1,1,1,0.0,0.25,1


In [20]:
basis = [(i, j, k) for i in [0, 1] for j in [0, 1] for k in [0, 1]]
for I in range(2 ** 3):
    dwstate = np.array([int(i) for i in '{0:05b}'.format(I)])
    print(I, basis[I], dwstate)

0 (0, 0, 0) [0 0 0 0 0]
1 (0, 0, 1) [0 0 0 0 1]
2 (0, 1, 0) [0 0 0 1 0]
3 (0, 1, 1) [0 0 0 1 1]
4 (1, 0, 0) [0 0 1 0 0]
5 (1, 0, 1) [0 0 1 0 1]
6 (1, 1, 0) [0 0 1 1 0]
7 (1, 1, 1) [0 0 1 1 1]
